# Full Text Search with Milvus

## Setup and Configuration

---

In [12]:
from typing import List

import ollama
from pymilvus import (
    MilvusClient,
    DataType,
    Function,
    FunctionType,
    AnnSearchRequest,
    RRFRanker,
)
from ollama import chat, ChatResponse

In [2]:
uri = "http://localhost:19530"
collection_name = "full_text_demo"
client = MilvusClient(uri=uri)

## Collection Setup for Full-Text Search

---

### Text Analysis Configuration

In [3]:
# Define tokenizer parameters for text analysis
analyzer_params = {"tokenizer": "standard", "filter": ["lowercase"]}

### Collection Schema and BM25 Function

In [4]:
schema = MilvusClient.create_schema()
schema.add_field(
    field_name="id",
    datatype=DataType.VARCHAR,
    is_primary=True,
    auto_id=True,
    max_length=100,
)
schema.add_field(
    field_name="content",
    datatype=DataType.VARCHAR,
    max_length=65535,
    analyzer_params=analyzer_params,
    enable_match=True,  # Enable text matching
    enable_analyzer=True,  # Enable text analysis
)
schema.add_field(field_name="sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR)
schema.add_field(
    field_name="dense_vector",
    datatype=DataType.FLOAT_VECTOR,
    dim=1024,  # Dimension for text-embedding-3-small
)
schema.add_field(field_name="metadata", datatype=DataType.JSON)

bm25_function = Function(
    name="bm25",
    function_type=FunctionType.BM25,
    input_field_names=["content"],
    output_field_names="sparse_vector",
)

schema.add_function(bm25_function)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': True}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535, 'enable_match': True, 'enable_analyzer': True, 'analyzer_params': '{"tokenizer":"standard","filter":["lowercase"]}'}}, {'name': 'sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>, 'is_function_output': True}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'metadata', 'description': '', 'type': <DataType.JSON: 23>}], 'enable_dynamic_field': False, 'functions': [{'name': 'bm25', 'description': '', 'type': <FunctionType.BM25: 1>, 'input_field_names': ['content'], 'output_field_names': ['sparse_vector'], 'params': {}}]}

### Indexing and Collection Creation

In [5]:
index_params = MilvusClient.prepare_index_params()
index_params.add_index(
    field_name="sparse_vector",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="BM25",
)
index_params.add_index(field_name="dense_vector", index_type="FLAT", metric_type="IP")

if client.has_collection(collection_name):
    client.drop_collection(collection_name)
client.create_collection(
    collection_name=collection_name,
    schema=schema,
    index_params=index_params,
)
print(f"Collection '{collection_name}' created successfully")

Collection 'full_text_demo' created successfully


### Insert Data

In [6]:
model_name = "bge-m3"


def get_embeddings(texts: List[str]) -> List[List[float]]:
    if not texts:
        return []

    response = ollama.embed(model=model_name, input=texts)
    embeddings = response["embeddings"]
    return embeddings


In [7]:
documents = [
    {
        "content": "Milvus is a vector database built for embedding similarity search and AI applications.",
        "metadata": {"source": "documentation", "topic": "introduction"},
    },
    {
        "content": "Full-text search in Milvus allows you to search using keywords and phrases.",
        "metadata": {"source": "tutorial", "topic": "full-text search"},
    },
    {
        "content": "Hybrid search combines the power of sparse BM25 retrieval with dense vector search.",
        "metadata": {"source": "blog", "topic": "hybrid search"},
    },
]

entities = []
texts = [doc["content"] for doc in documents]
embeddings = get_embeddings(texts)
print(f"Length of embeddings: {len(embeddings)}")

for i, doc in enumerate(documents):
    print(f"Embedding dimension: {len(embeddings[i])}")

Length of embeddings: 3
Embedding dimension: 1024
Embedding dimension: 1024
Embedding dimension: 1024


In [8]:
for i, doc in enumerate(documents):
    entities.append(
        {
            "content": doc["content"],
            "dense_vector": embeddings[i],
            "metadata": doc.get("metadata", {}),
        }
    )

client.insert(collection_name, entities)
print(f"Inserted {len(entities)} documents")

Inserted 3 documents


## Perform Retrieval

---

You can flexibly use the `search()` or `hybrid_search()` methods to implement full-text search (sparse), semantic search (dense), and hybrid search to lead to more robust and accurate search results.

### Full-Text Search

In [9]:
query = "full-text search keywords"

results = client.search(
    collection_name=collection_name,
    data=[query],
    anns_field="sparse_vector",
    limit=5,
    output_fields=["content", "metadata"],
)
sparse_results = results[0]

print("\nSparse Search (Full-text search):")
for i, result in enumerate(sparse_results):
    print(
        f"{i+1}. Score: {result['distance']:.4f}, Content: {result['entity']['content']}"
    )


Sparse Search (Full-text search):
1. Score: 3.1261, Content: Full-text search in Milvus allows you to search using keywords and phrases.
2. Score: 0.1836, Content: Hybrid search combines the power of sparse BM25 retrieval with dense vector search.
3. Score: 0.1335, Content: Milvus is a vector database built for embedding similarity search and AI applications.


### Semantic Search

In [10]:
query = "How does Milvus help with similarity search?"

query_embedding = get_embeddings([query])[0]

results = client.search(
    collection_name=collection_name,
    data=[query_embedding],
    anns_field="dense_vector",
    limit=5,
    output_fields=["content", "metadata"],
)
dense_results = results[0]

print("\nDense Search (Semantic):")
for i, result in enumerate(dense_results):
    print(
        f"{i+1}. Score: {result['distance']:.4f}, Content: {result['entity']['content']}"
    )


Dense Search (Semantic):
1. Score: 0.7608, Content: Milvus is a vector database built for embedding similarity search and AI applications.
2. Score: 0.6684, Content: Full-text search in Milvus allows you to search using keywords and phrases.
3. Score: 0.4786, Content: Hybrid search combines the power of sparse BM25 retrieval with dense vector search.


### Hybrid Search

In [11]:
query = "what is hybrid search"

query_embedding = get_embeddings([query])[0]

sparse_search_params = {"metric_type": "BM25"}
sparse_request = AnnSearchRequest(
    [query],
    "sparse_vector",
    sparse_search_params,
    limit=5
)

dense_search_params = {"metric_type": "IP"}
dense_request = AnnSearchRequest(
    [query_embedding],
    "dense_vector",
    dense_search_params,
    limit=5
)

results = client.hybrid_search(
    collection_name,
    [sparse_request, dense_request],
    ranker=RRFRanker(),  # Reciprocal Rank Fusion for combining results
    limit=5,
    output_fields=["content", "metadata"],
)
hybrid_results = results[0]

print("\nHybrid Search (Combined):")
for i, result in enumerate(hybrid_results):
    print(
        f"{i+1}. Score: {result['distance']:.4f}, Content: {result['entity']['content']}"
    )


Hybrid Search (Combined):
1. Score: 0.0328, Content: Hybrid search combines the power of sparse BM25 retrieval with dense vector search.
2. Score: 0.0323, Content: Milvus is a vector database built for embedding similarity search and AI applications.
3. Score: 0.0317, Content: Full-text search in Milvus allows you to search using keywords and phrases.


## Answer Generation

---

In [15]:
context = "\n\n".join([doc["entity"]["content"] for doc in hybrid_results])

prompt = f"""Answer the following question based on the provided context. 
If the context doesn't contain relevant information, just say "I don't have enough information to answer this question."

Context:
{context}

Question: {query}

Answer:"""

response: ChatResponse = chat(
    model='gemma3',
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers questions based on the provided context.",
        },
        {"role": "user", "content": prompt},
    ],
)

print(response.message.content)

Hybrid search combines the power of sparse BM25 retrieval with dense vector search.
